*prefix_encoder权重混合器*

本脚本旨在解决chatglm2-6b ptuning后遗忘原功能的问题


思路：
ptuning训练的权重本质是练model.transformer.prefix_encoder。
因此此脚本将老的prefix_encoder权重和ptuning训练的权重混合一下，再重新注入model.transformer.prefix_encoder中。

经过测试老权重0.2，新权重0.8，这样加权平均后的模型能打招呼并保留ptuning训练的能力
不过这种平均值混合权重的方法比较玄学，不一定有用。

In [ ]:
#基础环境载入
#地址输入
model_path = 
ptune_model_path=
origin_encoder_save_path='origin_prefix_encoder.pth'
mixed_encoder_save_path='mixed_prefix_encoder.pth'
#调参
origin_p=0.2#混合度(0-1),原始权重保留度

import os
import torch
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModel
from IPython.display import display, Markdown, clear_output

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def display_answer(model, query, history=[]):
    for response, history in model.stream_chat(
            tokenizer, query, history=history):
        clear_output(wait=True)
        display(Markdown(response))
    return history


config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=128)

In [ ]:
#读取原模型
model = AutoModel.from_pretrained(model_path,config=config,trust_remote_code=True)
model = model.cuda()
model = model.eval()
# 将模型的状态字典保存到文件中
torch.save(model.transformer.prefix_encoder.state_dict(), origin_encoder_save_path)

In [ ]:
#混合原prefix_encoder权重与adgn权重
o_prefix_encoder=dict(torch.load(origin_encoder_save_path))
myself_prefix_encoder = torch.load(os.path.join(ptune_model_path, "pytorch_model.bin"))
mixed_prefix_encoder={}
mixed_prefix_encoder['embedding.weight']=origin_p*o_prefix_encoder['embedding.weight']+(1-origin_p)*myself_prefix_encoder['transformer.prefix_encoder.embedding.weight']

In [ ]:
#保存混合权重
torch.save(mixed_prefix_encoder, mixed_encoder_save_path)

In [ ]:
#读取混合权重
mixed_path='mixed_prefix_encoder.pth'
mixed_prefix_encoder=torch.load(mixed_path)

In [ ]:
#注入混合权重
model.transformer.prefix_encoder.load_state_dict(mixed_prefix_encoder)


**选择运行**

存储整个大模型

In [ ]:
# 将模型的状态字典保存到文件中
torch.save(model.state_dict(), 'saved_model.pth')
#将saved_model.pth换掉model_path中的pytorch_model.bin使用

In [ ]:
#推理
history=display_answer(model, "more")